## ***Adaptations on the Explanations of Instance-based Methods***


> Moduled: Accpeting the four GNNs (GCN+GAP, DGCNN, DIFFPOOL, and GIN)


---

In [ ]:
"""
The algorithm:
Profile each edge's existance in all explanations by source and target node features.
Given a threshold, return saliency maps for each class by those of edges which satisfy the thresholding on frequency of appearance
on explanations of the target class.

This algorithms relaxes instance-specific explanations by ignoring the substructures in the explanations which are not frequently
happening on all instances of the entire class.
"""


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install git+https://github.com/rusty1s/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/rusty1s/pytorch_geometric.git to /tmp/pip-req-build-2ljov0g7
  Running command git clone --filter=blob:none --quiet https://github.com/rusty1s/pytorch_geometric.git /tmp/pip-req-build-2ljov0g7
  Resolved https://github.com/rusty1s/pytorch_geometric.git to commit 61c47ee404f8e26b3a1cd0db56448b6254920d0e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from math import sqrt
import math
from torch_geometric.datasets import TUDataset
import torch as th
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.parameter import Parameter
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, LayerNorm
from sklearn import metrics
from scipy.spatial.distance import hamming
import statistics
import pandas
from time import perf_counter
from IPython.core.display import deepcopy
from torch_geometric.nn import MessagePassing
import copy
from torch.nn import ReLU, Sequential
from torch import sigmoid
from itertools import chain
from time import perf_counter
from torch_geometric.data import Data, Batch, Dataset
from functools import partial
from torch_geometric.utils import to_networkx
from torch_geometric.utils import remove_self_loops
from typing import Callable, Union, Optional
#from torch_geometric.utils.num_nodes import maybe_num_nodes
import networkx as nx
from typing import List, Tuple, Dict
from collections import Counter
import statistics
import tqdm
import csv
from statistics import mean
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')

Processing...
Done!


In [ ]:
Explainability_name = 'TopDown Approach'
Task_name = 'Graph Classification'
checkpoint_directory_Classifier = "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/model_classifier.pt"
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
classifier_bias = True
DataSet_name = "MUTAG"

#File_Name = Model_name + " " + Explainability_name + " " + Task_name + " " + DataSet_name + " "

In [ ]:
#train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=0, shuffle=True)
#print("Number of Training Graphs: ", len(train_dataset))
#print("Number of Test Graphs: ", len(test_dataset))

df = pandas.read_csv("/content/drive/My Drive/Explainability Methods/Train and Test Indexes on Graph Classification/Experimental Results/train_test_indexes.csv")

read_training_list_indexes__ = df['Train Indexes']
read_test_list_indexes__ = df['Test Indexes']
read_test_list_indexes__ = read_test_list_indexes__.dropna()
read_test_list_indexes = []
read_training_list_indexes = []
for element in read_test_list_indexes__:
    read_test_list_indexes.append(int(element))
for element in read_training_list_indexes__:
    read_training_list_indexes.append(int(element))


print(read_training_list_indexes)
print(read_test_list_indexes)

mutag_train_dataset = []
mutag_test_dataset = []
for index in read_training_list_indexes:
    mutag_train_dataset.append(mutag_dataset[index])
for index in read_test_list_indexes:
    mutag_test_dataset.append(mutag_dataset[index])


print(f'Number of training graphs: {len(mutag_train_dataset)}')
print(f'Number of test graphs: {len(mutag_test_dataset)}')

[51, 142, 143, 10, 4, 141, 125, 23, 3, 79, 28, 117, 136, 156, 83, 128, 109, 70, 31, 58, 7, 148, 14, 187, 20, 162, 123, 13, 91, 185, 169, 102, 32, 55, 119, 25, 161, 175, 52, 121, 152, 108, 113, 65, 39, 103, 131, 42, 166, 110, 160, 68, 90, 89, 64, 172, 159, 72, 170, 18, 122, 29, 179, 49, 171, 178, 9, 74, 96, 48, 181, 127, 126, 87, 12, 163, 88, 53, 94, 146, 5, 158, 16, 67, 6, 59, 164, 151, 34, 47, 54, 46, 100, 112, 93, 182, 66, 106, 124, 19, 186, 133, 45, 15, 40, 167, 174, 98, 105, 153, 61, 63, 132, 116, 43, 80, 33, 147, 165, 69, 135, 86, 76, 57, 173, 115, 138, 140, 134, 180, 95, 22, 38, 41, 24, 120, 145, 26, 21, 50, 176, 107, 78, 17, 85, 154, 60, 92, 184, 129]
[0, 1, 2, 8, 11, 27, 30, 35, 36, 37, 44, 56, 62, 71, 73, 75, 77, 81, 82, 84, 97, 99, 101, 104, 111, 114, 118, 130, 137, 139, 144, 149, 150, 155, 157, 168, 177, 183]
Number of training graphs: 150
Number of test graphs: 38


In [ ]:
class profile_frequencies:
    def __init__(self, list_of_saliencies, test_dataset, style):
        self.list_of_saliencies = list_of_saliencies
        self.edges_profile_by_node_feats = {}
        self.edges_profile_by_node_feats_2_proportions = {}
        self.test_dataset = test_dataset
        self.style = style


        self.profile_edges_for_each_explanation()
        self.get_proportions()

        # for key, value in self.edges_profile_by_node_feats_2_proportions.items():
        #     print("Key: ", key, "      ", value)

    def profile_edges_for_each_explanation(self):
        for expl, graph in zip(self.list_of_saliencies, self.test_dataset):
            graph_edges = [(source_index, target_index) for source_index, target_index in graph.edge_index.T.tolist()]
            if self.style == "Edge":
                self.get_paired_features_as_keys_by_edge_saleincy(expl, graph_edges, graph.x)
            elif self.style == "Node":
                self.get_paired_features_as_keys_by_node_saleincy(expl, graph_edges, graph.x)

    def get_paired_features_as_keys_by_edge_saleincy(self, expl, edges, node_feats):
        for edge_sal, (source_index, target_index) in zip(expl, edges):
            if edge_sal == 1:
                edge_key = tuple([tuple(node_feats[source_index].tolist()), tuple(node_feats[target_index].tolist())])
                if edge_key in self.edges_profile_by_node_feats.keys():
                    self.edges_profile_by_node_feats[edge_key] = self.edges_profile_by_node_feats[edge_key] + 1
                else:
                    self.edges_profile_by_node_feats[edge_key] = 1

    def get_paired_features_as_keys_by_node_saleincy(self, expl, edges, node_feats):
        for (source_index, target_index) in edges:
            if (expl[source_index] == 1) and (expl[target_index] == 1):
                edge_key = tuple([tuple(node_feats[source_index].tolist()), tuple(node_feats[target_index].tolist())])
                if edge_key in self.edges_profile_by_node_feats.keys():
                    self.edges_profile_by_node_feats[edge_key] = self.edges_profile_by_node_feats[edge_key] + 1
                else:
                    self.edges_profile_by_node_feats[edge_key] = 1

    def get_proportions(self):
        sum_value = sum(self.edges_profile_by_node_feats.values())
        for key, value in self.edges_profile_by_node_feats.items():
            self.edges_profile_by_node_feats_2_proportions[key] = value/sum_value


    def profile_the_upcoming_explanation(self, an_explanation, a_graph):
        profile = []
        if self.style == "Edge":
            graph_edges = [(source_index, target_index) for source_index, target_index in a_graph.edge_index.T.tolist()]
            for sal_score, (source_index, target_index) in zip(an_explanation, graph_edges):
                edge_key = tuple([tuple(a_graph.x[source_index].tolist()), tuple(a_graph.x[target_index].tolist())])
                if sal_score == 1:
                    profile.append(self.edges_profile_by_node_feats_2_proportions[edge_key])
                else:
                    profile.append(0)
            return profile

        elif self.style == "Node":
            graph_edges = [(source_index, target_index) for source_index, target_index in a_graph.edge_index.T.tolist()]
            for (source_index, target_index) in graph_edges:
                edge_key = tuple([tuple(a_graph.x[source_index].tolist()), tuple(a_graph.x[target_index].tolist())])
                if (an_explanation[source_index] == 1) and (an_explanation[target_index] == 1):
                    profile.append(self.edges_profile_by_node_feats_2_proportions[edge_key])
                else:
                    profile.append(0)
            return profile

    def __call__(self, an_explanation, a_graph, threshold):

        profile = self.profile_the_upcoming_explanation(an_explanation, a_graph)
        profile_one_hot = [1 if value > threshold else 0 for value in profile]
        return profile_one_hot


edge_explanations = []
node_explanations = []
number_of_graphs = 10
for i in range(number_of_graphs):
    edge_explanations.extend(torch.randint(2, (1, mutag_test_dataset[i].edge_index[0].size()[0])).tolist())
    node_explanations.extend(torch.randint(2, (1, mutag_test_dataset[i].x.size()[0])).tolist())

print("edge_explanations: ", edge_explanations)
print("node_explanations: ", node_explanations)

common_edges_finder = profile_frequencies(list_of_saliencies=edge_explanations, test_dataset=mutag_test_dataset[:number_of_graphs], style="Edge")
graph_index = 0
profile_one_hot = common_edges_finder(edge_explanations[graph_index], mutag_test_dataset[graph_index], 0.7)
print(" input graph number of nodes: ", len(mutag_test_dataset[graph_index].x), " input graph number of edges: ", len(mutag_test_dataset[graph_index].edge_index[0]))
for i, value in enumerate(profile_one_hot):
    print(i, "   ", value)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     0
31     1
32     1
33     0
34     0
35     0
36     0
37     0


In [ ]:
sizes = []
for graph in mutag_dataset:
    sizes.append(len(graph.x))

In [ ]:
print(sizes.index(max(sizes)))
print(sizes.index(min(sizes)))

5
75
